<a href="https://colab.research.google.com/github/battuzz/torch_aot/blob/main/TorchAOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AOT Compilation of torch models

In [1]:
import torch

## Model definition

In [8]:
NUM_INPUTS = 5
NUM_OUTPUTS = 7
NUM_INDUCING_POINTS = 350

class ModelNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(NUM_INPUTS, 128)
        self.fc2 = torch.nn.Linear(128, 128)
        self.fc3 = torch.nn.Linear(128, NUM_OUTPUTS)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

def squared_distance(x1, x2):
    return (
        torch.sum(x1**2, dim=1, keepdim=True)
        + torch.sum(x2**2, dim=1)
        - 2 * torch.mm(x1, x2.t())
    )


def rbf_kernel(x1, x2, lengthscale=1.0):
    dist = squared_distance(x1 / lengthscale, x2 / lengthscale)
    return torch.exp(-0.5 * dist)

class ModelGPPosterior(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.lengthscales = torch.nn.Parameter(torch.randn(NUM_INPUTS))
        self.inducing_points = torch.nn.Parameter(
        torch.randn(NUM_INDUCING_POINTS, NUM_INPUTS)
        )
        self.alpha = torch.nn.Parameter(torch.randn(NUM_INDUCING_POINTS, NUM_OUTPUTS))

    def forward(self, x):
        Kuf = rbf_kernel(x, self.inducing_points, self.lengthscales)
        mean = Kuf @ self.alpha
        return mean

In [9]:
def train_with_random_data(model):
    X = torch.randn(1000, NUM_INPUTS)
    y = torch.randn(1000, NUM_OUTPUTS)

    model.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    for epoch in range(30):
        optimizer.zero_grad()
        output = model(X)
        loss = torch.nn.functional.mse_loss(output, y)
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

In [10]:
model_nn = ModelNN()
model_gp = ModelGPPosterior()
train_with_random_data(model_nn)
train_with_random_data(model_gp)

Epoch 1, Loss: 1.0171658992767334
Epoch 2, Loss: 1.016040325164795
Epoch 3, Loss: 1.0149654150009155
Epoch 4, Loss: 1.0139358043670654
Epoch 5, Loss: 1.0129508972167969
Epoch 6, Loss: 1.0120136737823486
Epoch 7, Loss: 1.0111228227615356
Epoch 8, Loss: 1.010277509689331
Epoch 9, Loss: 1.0094785690307617
Epoch 10, Loss: 1.0087215900421143
Epoch 11, Loss: 1.0080043077468872
Epoch 12, Loss: 1.0073307752609253
Epoch 13, Loss: 1.0066953897476196
Epoch 14, Loss: 1.006097674369812
Epoch 15, Loss: 1.0055360794067383
Epoch 16, Loss: 1.0050055980682373
Epoch 17, Loss: 1.0045056343078613
Epoch 18, Loss: 1.0040339231491089
Epoch 19, Loss: 1.0035897493362427
Epoch 20, Loss: 1.003167986869812
Epoch 21, Loss: 1.00276780128479
Epoch 22, Loss: 1.0023870468139648
Epoch 23, Loss: 1.0020263195037842
Epoch 24, Loss: 1.0016804933547974
Epoch 25, Loss: 1.0013495683670044
Epoch 26, Loss: 1.0010309219360352
Epoch 27, Loss: 1.0007250308990479
Epoch 28, Loss: 1.000428557395935
Epoch 29, Loss: 1.0001425743103027
E

In [13]:
example_input = torch.randn((1, NUM_INPUTS))
model_nn.eval()

exported = torch.export.export(model_nn, (example_input,))
torch._inductor.aoti_compile_and_package(
    exported,
    package_path="model_nn.pt2",
)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


'model_nn.pt2'

In [14]:
!pip install cmake

In [24]:
!chmod +x ./build.sh

In [27]:
!./build.sh

./build.sh: line 3: $'\r': command not found
./build.sh: line 4: $'\r': command not found
CMake Warning:
  Ignoring extra path from command line:

"


CMake Error at CMakeLists.txt:4 (find_package):
  By not providing "FindTorch.cmake" in CMAKE_MODULE_PATH this project has
  asked CMake to find a package configuration file provided by "Torch", but
  CMake did not find one.

  Could not find a package configuration file provided by "Torch" with any of
  the following names:

    TorchConfig.cmake
    torch-config.cmake

  Add the installation prefix of "Torch" to CMAKE_PREFIX_PATH or set
  "Torch_DIR" to a directory containing one of the above files.  If "Torch"
  provides a separate development package or SDK, be sure it has been
  installed.


-- Configuring incomplete, errors occurred!
gmake: Makefile: No such file or directory
gmake: *** No rule to make target 'Makefile'.  Stop.
./build.sh: line 9: $'\r': command not found
